# Phi-4 Mini ONNX並列関数呼び出しチュートリアル

このノートブックでは、Phi-4 MiniをONNX Runtime GenAIと組み合わせて並列関数呼び出しを行う方法を説明します。関数呼び出しを使用すると、モデルがユーザーのリクエストに基づいて外部ツールやAPIを賢く呼び出すことができます。

## 概要

このチュートリアルでは以下を学びます：
- Phi-4 MiniをONNX Runtime GenAIでセットアップする方法
- フライトやホテルの予約用の関数スキーマを定義する方法
- 構造化された出力のためにLark文法を使用したガイド付き生成
- 複雑な旅行予約シナリオで並列関数呼び出しを実行する方法

## 前提条件

このノートブックを実行する前に、以下を確認してください：
- Phi-4 Mini ONNXモデルをダウンロード済みであること
- `onnxruntime-genai`パッケージをインストール済みであること
- 関数呼び出しの基本概念を理解していること


## ステップ 1: 必要なライブラリをインポートする

まず、関数呼び出しの実装に必要なライブラリをインポートします。


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## ステップ 2: モデルのセットアップと構成

次に、Phi-4 Mini ONNXモデルを構成します。必ず実際のモデルディレクトリにパスを置き換えてください。


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## ステップ 3: 生成パラメータの設定

モデルの出力動作を制御するために生成パラメータを設定します。これらの設定により、関数呼び出しに対して決定論的で焦点を絞った応答が保証されます。


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## ステップ 4: 利用可能な関数を定義する

ここでは、AIアシスタントが呼び出せる関数を定義します。この例では、旅行関連の2つの関数があります：
1. **booking_flight_tickets**: 空港間のフライト予約用
2. **booking_hotels**: ホテル宿泊予約用

関数の定義は、OpenAIの関数呼び出しスキーマ形式に従っています。


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## ステップ5: 文法生成補助関数

これらの補助関数は、関数定義をLark文法形式に変換します。この形式は、ONNX Runtime GenAIによるガイド付き生成に使用されます。これにより、モデルが正しいJSON形式で有効な関数呼び出しを出力することが保証されます。


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## ステップ6: 文法生成のテスト

文法生成関数をテストして、ツール定義が正しい形式に変換されるか確認しましょう。


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## ステップ7: システムプロンプトとジェネレーターの準備

次に、利用可能なツールについてモデルに伝えるシステムプロンプトを作成し、ガイド付き生成パラメーターでジェネレーターを設定します。


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## ステップ 8: ガイド付き生成でジェネレーターを初期化する

次に、設定したパラメーターを使用してジェネレーターを作成し、ガイド付き生成のためにLark文法を適用します。


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## ステップ 9: 並列関数呼び出しのテスト

次に、複数の関数を呼び出す必要がある複雑な旅行予約シナリオで、設定をテストしてみましょう。


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### 関数呼び出しの生成

モデルは、ユーザーのリクエストに基づいて構造化された関数呼び出しを生成します。ガイド付き生成のおかげで、出力は直接実行可能な有効なJSON形式になります。

**期待される出力**: モデルは以下の関数呼び出しを生成する必要があります：
1. 北京（PEK）からパリ（CDG）への詳細を含む `booking_flight_tickets`
2. パリの宿泊施設の詳細を含む `booking_hotels`

以下のセルを実行してライブ生成を確認してください:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## 結論

🎉 **おめでとうございます！** Phi-4 Miniを使用したONNX Runtime GenAIによる並列関数呼び出しの実装に成功しました。

### 学んだこと:

1. **モデル設定**: Phi-4 MiniをONNX Runtime GenAIで設定する方法
2. **関数定義**: AI関数呼び出しのための関数スキーマを定義する方法
3. **ガイド付き生成**: Lark文法を使用して構造化された出力を生成する方法
4. **並列関数呼び出し**: 複数の関数呼び出しを必要とする複雑なリクエストを処理する方法

### 主なメリット:

- ✅ **構造化された出力**: ガイド付き生成により有効なJSON関数呼び出しを保証
- ✅ **並列処理**: 1つのリクエストで複数の関数呼び出しを処理
- ✅ **高性能**: ONNX Runtimeが最適化された推論を提供
- ✅ **柔軟なスキーマ**: 関数定義の追加や変更が簡単

### リソース:

- [Phi-4 Mini ドキュメント](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI ドキュメント](https://onnxruntime.ai/docs/genai/)
- [関数呼び出しのベストプラクティス](https://platform.openai.com/docs/guides/function-calling)



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当社は責任を負いません。
